In [50]:
# libraries :>

# libraries :> 

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager as CM
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pynput
import time

import cv2 as cv
import numpy as np

import ollama

In [62]:
# Helpers :> 
def drawGridLabel(img):
    h, w, _ = img.shape
    grid_shape = (10,20)
    rows, cols = grid_shape
    dy, dx = h / rows, w / cols
    color = (0, 0, 0)
    color = (0, 255, 0)
    thickness = 1
    # draw vertical lines
    for x in np.linspace(start=dx, stop=w-dx, num=cols-1):
        x = int(round(x))
        cv.line(img, (x, 0), (x, h), color=color, thickness=thickness)
    
    # draw horizontal lines
    for y in np.linspace(start=dy, stop=h-dy, num=rows-1):
        y = int(round(y))
        cv.line(img, (0, y), (w, y), color=color, thickness=thickness)

    # Increasing the brighness :>
    # img = increase_brightness(img,value=100)
    
    ## Adding numbering the box :> 
    i = 0
    font = cv.FONT_HERSHEY_SIMPLEX
    # fontScale
    fontScale = 0.8
    thickness =1
    color = (0, 255, 0)
    for x in np.linspace(start=0, stop=w-dx, num=cols):
        x = int(round(x))
        for y in np.linspace(start=dy, stop=h, num=rows):
            i+=1
            y = int(round(y))
            mid_point = x + 10, y - 25
            # print(mid_point)
            img = cv.putText(img, str(i), mid_point, font, fontScale, color, thickness, cv.LINE_AA)
    return img
    

In [65]:
# Initializations :> 
# Initating Chrome web browser
chrome_options = Options()
# Use WebDriverManager to handle driver installation
service = Service(executable_path=CM().install())
driver = webdriver.Chrome(service=service, options=chrome_options)


# Steps :> 
# 1
# Navigate to login page
url = "https://www.google.com/finance/?hl=en"
driver.get(url)

image_file_name = "page.png"
driver.get_screenshot_as_file(image_file_name)

# 2
img = cv.imread(image_file_name)
# Grayscale 
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 
# Find Canny edges 
edged = cv.Canny(gray, 10, 100) 
contours, hierarchy = cv.findContours(edged,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE) 

areas = [cv.contourArea(contour) for contour in contours]
areas_sorted = sorted(areas,reverse=True)

## Getting top 10 area contours :>
top = 5
top_contours = []
for i in range(top):
    ind = areas.index(areas_sorted[i])
    top_contours.append(contours[ind])
    
ind = 0
for i in top_contours:
    M = cv.moments(i)
    ind += 1
    if M['m00'] != 0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        cv.drawContours(img, [i], -1, (0, 255, 0), 2)
        # cv.circle(img, (cx, cy), 7, (0, 0, 255), -1)
        cv.putText(img, str(ind), (cx , cy ), cv.FONT_HERSHEY_SIMPLEX, 1 ,(0, 255, 0), 2)

img = cv.drawContours(img, top_contours, -1, (0, 255, 0), 3) 
cv.imwrite("contours.png", img)

prompt = """Role: You are an Expert Visual Analyzer tool. 
You are given an image with labelled green boxes around the objects. These green boxes or contours are numerically labelled from 1, 2, and so on. 
Identify which box has the given below target object. Give me numerical label of the contour/box.

Target object: Search bar with text - Search for stocks, ETFs & more

Task:
    You need to indetify which labelled contours where target object is located. 
    
Explanation:
    You are supposed to idenitify the exaction controus of the target object specified above. 
    A contours is present with labbelling of each contour. You need to provide label of the contour which has the target object in it. 

Additional Information: 
Contours and its labels are in green color

Steps: 
    1. Analyze whether specified target object is present in the Webpage image. 
    2. Get the reference labbeled box of the same target object location.
    3. Give me the label of the box which has the target object in it. 

Give me only the label number of the contour which has the target object. Don't provide any other information. 
"""

res = ollama.chat(
	model="llava:7b",
	messages=[
		{
			'role': 'user',
			'content': prompt,
			'images': [ './contours.png']
		}
	]
)

response = res['message']['content']

# Extract number
import re
ind = int(re.findall(r'\d+',response)[0])

M = cv.moments(top_contours[ind])

if M['m00'] != 0:
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
center = (cx,cy)

offset = (-270, 50)

# new center :> 
center_x = center[0] + offset[0]
center_y = center[1] + offset[1]
new_center = (center_x,center_y)

mouse = pynput.mouse.Controller()
mouse.position = new_center

# Press and release
time.sleep(1)
mouse.press(pynput.mouse.Button.left)
mouse.release(pynput.mouse.Button.left)


keyboard = pynput.keyboard.Controller()

search = "apple"
keyboard.type(search)

time.sleep(1)
keyboard.press(pynput.keyboard.Key.enter)
keyboard.release(pynput.keyboard.Key.enter)
